# 유방암의 임파선 전이 예측 AI 경진 대회
https://dacon.io/competitions/official/236011/talkboard/407176?page=1&dtype=recent
## 배경지식
+ 림프절은 암의 전이, 퍼지는데 치명적인 원인
+ 림프절 전이 여부에 따라 치료와 예후가 달라짐
+ 림프절 전이 여부와 전이 단계 파악이 핵심적인 역할
## 제공 데이터
+ 유방암 병리 슬라이드 영상
+ 임상 항목 데이터
## 주제
+ 유방암 병리 슬라이드 영상과 임상 항목을 조합하여 유방암의 임파선 전이 여부 이진 분류
## 데이터 이해
+ ID : 부여번호
+ img_path 
+ mask_path
+ 나이 : 진단시 나이
+ 수술연월일 : 연-월-일
+ 진단명 : 1-ductal, 2-lobular, 3-mucinous, 4-other
+ 암의 위치 : 1-right, 2-left, 3-both
+ 암의 개수 : 1-single, 2-multiple
+ NG : 핵의 모양 ,1-핵모양 정상, 2-중간, 3-이상
+ HG : 1-(scores of 3, 4, or 5): 조직학적 분화도가 좋아 전체적으로 합산 점수가 낮은 경우,
2-(scores of 6 or 7), 3-(scores of 8 or 9) : 조직학적 분화도가 나빠서 합산점수가 큰 경우  
4: Only microinvasion present (not graded)
+ HG_score_1 : 
+ HG_score_2
+ HG_score_3
+ DCIS or LCIS 여부 : 제자리암종(DCIS) 유무
+ DCIS or LCIS_type : 제자리암종내 괴사 유무
+ T_category
+ ER : 여성호르몬중 에스트로겐 수용체 발현 여부
+ ER_Allred_score
+ PR : 여성호르몬 중 프로게스테론 수용체 발현 여부
+ PR_Allred_score 
+ KI-67_LI_percent : 세포증식 수치
+ HER2 : 세포성장, 수용체 발현여부 표시 0- -, 1- +
+ HER2_IHC
+ HER2_SISH : 
+ HER2_SISH_ratio : 소수점 첫째자리 숫자 기입
+ N_category : 0-전이 X, 1-전이 O




In [1]:
import os 
import io
import zipfile
from zipfile import ZipFile
import pandas as pd
import requests
import tensorflow as tf
import cv2

### 데이터 로드 과정

In [2]:
with zipfile.ZipFile('/content/drive/MyDrive/Project/유방암/Data/open.zip', 'r') as existing_zip:
    existing_zip.extractall('/content/drive/MyDrive/Project/유방암/Data/')

FileNotFoundError: ignored

### 데이터 확인

In [3]:
data=pd.read_csv("/content/drive/MyDrive/Project/유방암/Data/train.csv")

In [4]:
data.head()

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN,0


In [5]:
data.corr()

,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,HG_score_3,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
나이,1.000000,-0.025382,0.030878,-0.111352,0.127918,0.041033,0.029207,0.079623,0.033432,-0.011529,...,0.205562,-0.183891,0.038598,-0.025812,0.008067,0.023104,0.077032,0.014711,0.112657,0.069017
진단명,-0.025382,1.000000,-0.011532,0.090882,0.006628,-0.176999,-0.182840,-0.268685,-0.226373,-0.138698,...,0.055227,0.030321,0.026178,-0.095352,-0.077823,-0.021894,-0.111528,-0.069496,-0.073626,-0.049135
암의 위치,0.030878,-0.011532,1.000000,0.002039,-0.005704,0.062808,0.042752,0.023241,0.030181,0.053286,...,0.018644,-0.003590,0.042384,0.036607,-0.019287,-0.022304,0.066809,-0.090225,0.116714,-0.026330
암의 개수,-0.111352,0.090882,0.002039,1.000000,0.011457,0.090424,0.032388,0.094028,0.058740,-0.064365,...,-0.022538,0.040935,0.025864,0.037787,-0.121923,0.013808,0.032223,-0.069166,0.035007,0.268900
암의 장경,0.127918,0.006628,-0.005704,0.011457,1.000000,0.311455,0.353011,0.249808,0.309582,0.308494,...,0.080298,-0.101581,0.062952,0.237337,-0.002667,-0.015271,0.112631,0.348833,-0.078003,0.292102
NG,0.041033,-0.176999,0.062808,0.090424,0.311455,1.000000,0.759134,0.450426,0.979197,0.507472,...,-0.051401,-0.289941,-0.015848,0.517741,0.154768,0.170879,0.237103,0.134898,0.048978,0.371070
HG,0.029207,-0.182840,0.042752,0.032388,0.353011,0.759134,1.000000,0.645320,0.759587,0.726143,...,-0.066178,-0.295554,0.025963,0.550921,0.130307,0.116952,0.212581,0.158679,-0.008124,0.247525
HG_score_1,0.079623,-0.268685,0.023241,0.094028,0.249808,0.450426,0.645320,1.000000,0.429171,0.283791,...,0.022972,-0.155470,0.020299,0.295521,0.037352,0.032845,0.221743,0.087846,-0.216455,0.272786
HG_score_2,0.033432,-0.226373,0.030181,0.058740,0.309582,0.979197,0.759587,0.429171,1.000000,0.520825,...,-0.028491,-0.267796,0.003563,0.506303,0.121347,0.148694,0.232563,0.131684,0.039375,0.287524
HG_score_3,-0.011529,-0.138698,0.053286,-0.064365,0.308494,0.507472,0.726143,0.283791,0.520825,1.000000,...,-0.190123,-0.271423,-0.000629,0.563017,0.162601,0.108900,0.067702,0.217068,0.036129,0.047555


In [6]:
data.isna().sum()

ID                     0
img_path               0
mask_path              0
나이                     0
수술연월일                  0
진단명                    0
암의 위치                  0
암의 개수                  0
암의 장경                 69
NG                    51
HG                    86
HG_score_1            92
HG_score_2            92
HG_score_3            89
DCIS_or_LCIS_여부        0
DCIS_or_LCIS_type    874
T_category             4
ER                     1
ER_Allred_score      297
PR                     1
PR_Allred_score      453
KI-67_LI_percent     235
HER2                  14
HER2_IHC              24
HER2_SISH            753
HER2_SISH_ratio      825
BRCA_mutation        945
N_category             0
dtype: int64

In [7]:
data=data.fillna(0)

In [8]:
from PIL import Image
import numpy as np

In [9]:
data.img_path[0].split(".")[1]

'/train_imgs/BC_01_0001'

In [14]:
image=cv2.imread("/content/drive/MyDrive/Project/유방암/Data"+data.img_path[0].split(".")[1]+".png")

In [21]:
image=cv2.resize(255-image,(28,28))

In [22]:
image=np.array(image)

In [23]:
image.shape

(28, 28, 3)

In [24]:
data['images']='0'

In [25]:
data['images'][0]=image

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
for i in range(len(data)):
  try:
    image=cv2.imread("/content/drive/MyDrive/Project/유방암/Data"+data.img_path[i].split(".")[1]+".png")
    image=cv2.resize(255-image,(28,28))
    image=np.array(image)
    data['images'][i]=image
  except Exception as e:
    data['images'][i]='0'
    pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
data

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,images
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,1.0,6.0,12.0,0.0,1.0,0.0,0.00,0.0,0,"[[[11, 11, 11], [11, 11, 11], [11, 11, 11], [1..."
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,0.0,0.0,70.0,0.0,0.0,0.0,0.00,0.0,1,"[[[11, 11, 11], [11, 11, 11], [11, 11, 11], [1..."
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,0.0,2.0,...,1.0,4.0,7.0,0.0,1.0,0.0,0.00,0.0,0,"[[[11, 11, 11], [11, 11, 11], [11, 11, 11], [1..."
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,0.0,0.0,1.0,1.0,3.0,0.0,0.00,0.0,0,"[[[11, 11, 11], [11, 11, 11], [11, 11, 11], [1..."
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,0.0,0.0,8.0,1.0,2.0,1.0,5.44,0.0,0,"[[[11, 11, 11], [11, 11, 11], [11, 11, 11], [1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,0
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,0
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0


In [46]:
y=data.N_category

In [36]:
data.to_csv("/content/drive/MyDrive/Project/유방암/Data/train+image.csv")

In [41]:
data=data[data.images!='0']

/usr/local/lib/python3.7/dist-packages/pandas/core/ops/array_ops.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [58]:
X=data.images

In [43]:
from sklearn.model_selection import train_test_split

In [47]:
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=2022)

In [48]:
from tensorflow import keras

In [50]:
x_train.shape

(548,)

In [53]:


il=keras.layers.Input(shape=(548,))
hl=keras.layers.Dense(256,activation="relu")(il)
hl=keras.layers.Dense(256,activation="relu")(hl)
ol=keras.layers.Dense(28,activation="sigmoid")(hl)

model=keras.models.Model(inputs=il,outputs=ol)

model.compile(loss=keras.losses.binary_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 548)]             0         
                                                                 
 dense (Dense)               (None, 256)               140544    
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 28)                7196      
                                                                 
Total params: 213,532
Trainable params: 213,532
Non-trainable params: 0
_________________________________________________________________


In [54]:
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping

In [55]:
es=EarlyStopping(monitor='val_loss',
                 min_delta=0,
                 patience=3,
                 verbose=1,
                 restore_best_weights=True)

In [63]:
model.fit(x_train,y_train,epochs=50,validation_split=0.2,callbacks=[es])

ValueError: ignored